## 운항시즌 데이터 (SFSNT.CSV)

- SSC : 시즌코드
- FLT : 편명
- ORG : 출발공항
- DES : 도착공항
- STA : 출발시간
- STD : 도착시간
- FLO : 항공사
- MON ~ SUN : 월 ~ 일
- FSD : 시작일자
- FED : 종료일자
- IRR : 부정기타입 (시기나 기한이 일정하게 정해져 있지 아니함)

In [23]:
import numpy as np
import pandas as pd
from datetime import datetime

In [30]:
# 데이터 적재
season = pd.read_csv('/Users/Mac/Desktop/빅콘테스트/2019빅콘테스트_퓨처스리그/SFSNT.CSV')

In [31]:
season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 17 columns):
SSC    1171 non-null object
FLT    1171 non-null object
ORG    1171 non-null object
DES    1171 non-null object
STD    1153 non-null object
STA    1152 non-null object
FLO    1171 non-null object
MON    750 non-null object
TUE    745 non-null object
WED    739 non-null object
THU    741 non-null object
FRI    744 non-null object
SAT    743 non-null object
SUN    722 non-null object
FSD    1171 non-null int64
FED    1171 non-null int64
IRR    1171 non-null object
dtypes: int64(2), object(15)
memory usage: 155.6+ KB


In [32]:
season.describe(include='object')

,SSC,FLT,ORG,DES,STD,STA,FLO,MON,TUE,WED,THU,FRI,SAT,SUN,IRR
count,1171,1171,1171,1171,1153,1152,1171,750,745,739,741,744,743,722,1171
unique,1,619,15,15,180,184,8,1,1,1,1,1,1,1,2
top,S19,J1267,ARP3,ARP3,20:00,21:05,J,Y,Y,Y,Y,Y,Y,Y,N
freq,1171,9,434,425,23,29,547,750,745,739,741,744,743,722,1163


In [33]:
season.head()

# SSC는 필요없음 
# ORG, DES 공항은 15종류 
# FLO: 항공사는 8개 
# 부정기 타입은 Y: 8, N: 1163

,SSC,FLT,ORG,DES,STD,STA,FLO,MON,TUE,WED,THU,FRI,SAT,SUN,FSD,FED,IRR
0,S19,A1915,ARP1,ARP3,7:55,9:05,A,Y,Y,Y,Y,Y,Y,Y,20190331,20191026,N
1,S19,A1904,ARP3,ARP1,7:55,9:05,A,Y,Y,Y,Y,Y,Y,Y,20190331,20191026,N
2,S19,I1304,ARP3,ARP1,8:00,9:15,I,Y,NaN,Y,Y,Y,NaN,Y,20190331,20191026,N
3,S19,B1802,ARP2,ARP1,8:00,8:55,B,Y,Y,Y,Y,Y,Y,Y,20190331,20191026,N
4,S19,J1106,ARP1,ARP2,8:00,9:05,J,NaN,Y,Y,NaN,Y,Y,NaN,20190730,20190824,N


### Null 확인

In [26]:
season.isnull().sum()

SSC      0
FLT      0
ORG      0
DES      0
STD     18
STA     19
FLO      0
MON    421
TUE    426
WED    432
THU    430
FRI    427
SAT    428
SUN    449
FSD      0
FED      0
IRR      0
dtype: int64

In [27]:
missing_df = season.isnull().sum().reset_index()
missing_df.columns = ['column', 'count']
missing_df['ratio'] = missing_df['count'] / season.shape[0]
missing_df.loc[missing_df['ratio'] != 0]

,column,count,ratio
4,STD,18,0.015371
5,STA,19,0.016225
7,MON,421,0.359522
8,TUE,426,0.363792
9,WED,432,0.368915
10,THU,430,0.367208
11,FRI,427,0.364646
12,SAT,428,0.365500
13,SUN,449,0.383433


### 시계열 데이터로 변환

In [5]:
# FSD, FED --> 날짜
season['FSD'] = season.FSD.apply(lambda x: datetime.strptime(str(x),'%Y%m%d'))
season['FED'] = season.FED.apply(lambda x: datetime.strptime(str(x),'%Y%m%d'))

In [6]:
# STD, STA --> 시간
season.STA = season.STA.apply(lambda x: str(0)+x if len(str(x))==4 else x)
season.STD = season.STD.apply(lambda x: str(0)+x if len(str(x))==4 else x)
#season.STA.apply(lambda x: len(season.STA[x]) != 5)
#season['STA'] = season.STA.apply(lambda x: datetime.strptime(str(x),'%H%M'))

5

In [7]:
#season['lenSTA'] = [len(str(season.STA[i])) for i in range(len(season.STA))]
#season['lenSTD'] = [len(str(season.STD[i])) for i in range(len(season.STD))]

In [8]:
#season.STA[season.lenSTA != 5] # 아... 결측치때문이었군....
#season.STD[season.lenSTD != 5]
season.STA = season.STA.fillna('00:00')
season.STD = season.STD.fillna('00:00')

In [10]:
season['STA'] = season.STA.apply(lambda x: datetime.strptime(x,'%H:%M'))
season['STD'] = season.STD.apply(lambda x: datetime.strptime(x,'%H:%M'))

In [11]:
# STD, STA, FSD, FED를 dataetime으로 변환 완료!
season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 19 columns):
SSC       1171 non-null object
FLT       1171 non-null object
ORG       1171 non-null object
DES       1171 non-null object
STD       1171 non-null datetime64[ns]
STA       1171 non-null datetime64[ns]
FLO       1171 non-null object
MON       750 non-null object
TUE       745 non-null object
WED       739 non-null object
THU       741 non-null object
FRI       744 non-null object
SAT       743 non-null object
SUN       722 non-null object
FSD       1171 non-null datetime64[ns]
FED       1171 non-null datetime64[ns]
IRR       1171 non-null object
lenSTA    1171 non-null int64
lenSTD    1171 non-null int64
dtypes: datetime64[ns](4), int64(2), object(13)
memory usage: 173.9+ KB


In [12]:
season.head() # 앞에 년도 왜 나오지 ㅠ

,SSC,FLT,ORG,DES,STD,STA,FLO,MON,TUE,WED,THU,FRI,SAT,SUN,FSD,FED,IRR,lenSTA,lenSTD
0,S19,A1915,ARP1,ARP3,1900-01-01 07:55:00,1900-01-01 09:05:00,A,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,5,5
1,S19,A1904,ARP3,ARP1,1900-01-01 07:55:00,1900-01-01 09:05:00,A,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,5,5
2,S19,I1304,ARP3,ARP1,1900-01-01 08:00:00,1900-01-01 09:15:00,I,Y,NaN,Y,Y,Y,NaN,Y,2019-03-31,2019-10-26,N,5,5
3,S19,B1802,ARP2,ARP1,1900-01-01 08:00:00,1900-01-01 08:55:00,B,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,5,5
4,S19,J1106,ARP1,ARP2,1900-01-01 08:00:00,1900-01-01 09:05:00,J,NaN,Y,Y,NaN,Y,Y,NaN,2019-07-30,2019-08-24,N,5,5


In [13]:
#season.info()
#season.STD['1900-01-01 00:00:00' : '1900-01-01 06:59:59']  흠.. 왜 슬라이싱이 안될까요,,,,


### 가설1)  운항 period가 지연에 영향 미칠 것

In [14]:
# period
season['period'] = [(season.FED[i] - season.FSD[i]) for i in range(len(season.FSD))]
season.period.sort_values(ascending = False)

1170   209 days
615    209 days
612    209 days
611    209 days
609    209 days
608    209 days
601    209 days
599    209 days
589    209 days
588    209 days
586    209 days
584    209 days
583    209 days
582    209 days
580    209 days
579    209 days
577    209 days
563    209 days
560    209 days
613    209 days
616    209 days
558    209 days
622    209 days
641    209 days
639    209 days
637    209 days
636    209 days
635    209 days
634    209 days
633    209 days
         ...   
481      5 days
292      5 days
825      4 days
277      4 days
848      4 days
155      4 days
138      3 days
336      3 days
414      3 days
260      3 days
235      3 days
445      3 days
468      3 days
165      2 days
770      2 days
503      1 days
603      1 days
286      1 days
572      1 days
121      0 days
293      0 days
541      0 days
97       0 days
92       0 days
442      0 days
697      0 days
246      0 days
385      0 days
78       0 days
276      0 days
Name: period, Length: 11

In [15]:
season.head()

,SSC,FLT,ORG,DES,STD,STA,FLO,MON,TUE,WED,THU,FRI,SAT,SUN,FSD,FED,IRR,lenSTA,lenSTD,period
0,S19,A1915,ARP1,ARP3,1900-01-01 07:55:00,1900-01-01 09:05:00,A,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,5,5,209 days
1,S19,A1904,ARP3,ARP1,1900-01-01 07:55:00,1900-01-01 09:05:00,A,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,5,5,209 days
2,S19,I1304,ARP3,ARP1,1900-01-01 08:00:00,1900-01-01 09:15:00,I,Y,NaN,Y,Y,Y,NaN,Y,2019-03-31,2019-10-26,N,5,5,209 days
3,S19,B1802,ARP2,ARP1,1900-01-01 08:00:00,1900-01-01 08:55:00,B,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,5,5,209 days
4,S19,J1106,ARP1,ARP2,1900-01-01 08:00:00,1900-01-01 09:05:00,J,NaN,Y,Y,NaN,Y,Y,NaN,2019-07-30,2019-08-24,N,5,5,25 days


In [16]:
season.describe(include='object') # MON~ SUN: NAN은 빠지고 카운트 된건가?

,SSC,FLT,ORG,DES,FLO,MON,TUE,WED,THU,FRI,SAT,SUN,IRR
count,1171,1171,1171,1171,1171,750,745,739,741,744,743,722,1171
unique,1,619,15,15,8,1,1,1,1,1,1,1,2
top,S19,J1267,ARP3,ARP3,J,Y,Y,Y,Y,Y,Y,Y,N
freq,1171,9,434,425,547,750,745,739,741,744,743,722,1163


### 가설 2) 월~금 출항 횟수가 지연에 영향 미칠 것 

In [17]:
def myfunc(x):
    if x == 'Y':
        y = 1
    else:
        y = 0
    return y

season.MON = season.MON.apply(myfunc)
season.TUE = season.TUE.apply(myfunc)
season.WED = season.WED.apply(myfunc)
season.THU = season.THU.apply(myfunc)
season.FRI = season.FRI.apply(myfunc)
season.SAT = season.SAT.apply(myfunc)
season.SUN = season.SUN.apply(myfunc)
season['sum'] = season[['MON','TUE','WED','THU','FRI','SAT','SUN']].sum(axis=1)

In [18]:
season.head()

,SSC,FLT,ORG,DES,STD,STA,FLO,MON,TUE,WED,...,FRI,SAT,SUN,FSD,FED,IRR,lenSTA,lenSTD,period,sum
0,S19,A1915,ARP1,ARP3,1900-01-01 07:55:00,1900-01-01 09:05:00,A,1,1,1,...,1,1,1,2019-03-31,2019-10-26,N,5,5,209 days,7
1,S19,A1904,ARP3,ARP1,1900-01-01 07:55:00,1900-01-01 09:05:00,A,1,1,1,...,1,1,1,2019-03-31,2019-10-26,N,5,5,209 days,7
2,S19,I1304,ARP3,ARP1,1900-01-01 08:00:00,1900-01-01 09:15:00,I,1,0,1,...,1,0,1,2019-03-31,2019-10-26,N,5,5,209 days,5
3,S19,B1802,ARP2,ARP1,1900-01-01 08:00:00,1900-01-01 08:55:00,B,1,1,1,...,1,1,1,2019-03-31,2019-10-26,N,5,5,209 days,7
4,S19,J1106,ARP1,ARP2,1900-01-01 08:00:00,1900-01-01 09:05:00,J,0,1,1,...,1,1,0,2019-07-30,2019-08-24,N,5,5,25 days,4


In [19]:
season['sum'].value_counts()

7    435
1    220
2    146
3    105
4     95
5     88
6     82
Name: sum, dtype: int64

### 가설 3) 부정기 타입이 지연에 영향 미칠 것 

In [20]:
# 부정기 = Y 
# 부정기 타입이면 무엇이 특별한가?? 
season.IRR.value_counts()
# Y: 8, N: 1163
#season[season.IRR == 'Y']

N    1163
Y       8
Name: IRR, dtype: int64

1. 데이터를 임포트하여 메모리에 올린다.

2. 데이터의 모양을 확인 한다.

3. 데이터의 타입을 확인한다.

4. 데이터의 Null 값을 체크한다. 

5. 종속변수의 분포를 살펴본다.

6. 독립변수 - 명목형 변수의 분포를 살펴본다. 

7. 독립변수 - 수치형 변수의 분포를 살펴본다. 

8. 수치형, 명목형 변수간의 관계를 파악한다. 